In [35]:
# Reference : https://github.com/golbin/TensorFlow-Tutorials
import tensorflow as tf
import numpy as np

In [36]:
# [털, 날개]
x_data = np.array([[0,0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])
y_data = np.array([
    [1, 0, 0], # 기타
    [0, 1, 0], # 포유류
    [0, 0, 1], # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

In [37]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W = tf.Variable(tf.random_uniform([2, 3], -1.0, 1.0))
# 편향은 아웃풋 개수로 설정해야함
b = tf.Variable(tf.zeros([3]))

L = tf.add(tf.matmul(X, W), b)
L = tf.nn.relu(L)

# softmax 사용하는 이유

softmax 사용시 전체합이 1인 확률로 만들어줌
[8.04, 2.76, -6.52] -> [0.53 0.24 0.23]

In [38]:
model = tf.nn.softmax(L)

# axix =1 사용한 이유

전체 합이 아닌, 개별 결과를 구하고 평균 내는 방식을 사용하기 위해 axis 옵션을 사용함

axis 옵션이 없으면 -1.09 처럼 총합인 스칼라값으로 출력됨
```
        Y         model         Y * tf.log(model)   reduce_sum(axis=1)
 예) [[1 0 0]  [[0.1 0.7 0.2]  -> [[-1.0  0    0]  -> [-1.0, -0.09]
     [0 1 0]]  [0.2 0.8 0.0]]     [ 0   -0.09 0]]
```
즉, 이것은 예측값과 실제값 사이의 확률 분포의 차이를 비용으로 계산한 것이며,
이것을 Cross-Entropy 라고 함

In [39]:
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(model), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.8)
train_op = optimizer.minimize(cost)

# 신경망 모델 학습

In [40]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    if (step+1)  % 10 == 0:
        print(step + 1, 'cost :', sess.run(cost, feed_dict={X: x_data, Y: y_data}))

10 cost : 1.0076135
20 cost : 0.98803806
30 cost : 0.97806406
40 cost : 0.97082466
50 cost : 0.9616797
60 cost : 0.9556482
70 cost : 0.9510577
80 cost : 0.94824386
90 cost : 0.9448332
100 cost : 0.9449665


# 결과 확인

In [42]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도 : %.2f' % sess.run(accuracy*100, feed_dict = {X: x_data, Y: y_data}))

예측값: [0 1 0 0 0 0]
실제값: [0 1 2 0 0 2]
정확도 : 66.67
